In [35]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,OneHotEncoder,add_dummy_feature
import re
import collections
import cPickle as cp
import csv

#import string as str

In [6]:
rsc = pd.read_csv('/home/nishant/Downloads/Data/regular_season_compact_results.csv')
rsc.head()

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [5]:
rsd=pd.read_csv('/home/nishant/Downloads/Data/regular_season_detailed_results.csv')
rsd.head()

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot,wfgm,wfga,...,lfga3,lftm,lfta,lor,ldr,last,lto,lstl,lblk,lpf
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [7]:
tourc=pd.read_csv('/home/nishant/Downloads/Data/tourney_compact_results.csv')
tourc.head()
champlist=tourc[tourc['daynum']==154]['wteam'].tolist()
print champlist

[1437, 1257, 1231, 1242, 1276, 1424, 1181, 1181, 1314, 1116, 1417, 1246, 1112, 1246, 1163, 1277, 1181, 1268, 1393, 1163, 1314, 1196, 1196, 1242, 1314, 1181, 1163, 1246, 1257, 1163]


In [8]:
tourd=pd.read_csv('/home/nishant/Downloads/Data/tourney_detailed_results.csv')
tourd.head()

,season,daynum,wteam,wscore,lteam,lscore,wloc,numot,wfgm,wfga,...,lfga3,lftm,lfta,lor,ldr,last,lto,lstl,lblk,lpf
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


In [9]:
seed=pd.read_csv('/home/nishant/Downloads/Data/tourney_seeds.csv')
seed.head()

,season,seed,team
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [10]:
seed15=pd.read_csv('/home/nishant/Downloads/Data/tourney_seeds_2015.csv')
seed15.head()

,season,seed,team
0,2015,W01,1437
1,2015,W02,1438
2,2015,W03,1328
3,2015,W04,1257
4,2015,W05,1320


In [11]:
slot=pd.read_csv('/home/nishant/Downloads/Data/tourney_slots.csv')
slot.head()

,season,slot,strongseed,weakseed
0,1985,R1W1,W01,W16
1,1985,R1W2,W02,W15
2,1985,R1W3,W03,W14
3,1985,R1W4,W04,W13
4,1985,R1W5,W05,W12


In [12]:
slot15=pd.read_csv('/home/nishant/Downloads/Data/tourney_slots_2015.csv')
slot15.head()

,season,slot,strongseed,weakseed
0,2015,W11,W11a,W11b
1,2015,X16,X16a,X16b
2,2015,Y16,Y16a,Y16b
3,2015,Z11,Z11a,Z11b
4,2015,R1W1,W01,W16


In [13]:
teams=pd.read_csv('/home/nishant/Downloads/Data/teams.csv')
teamlist=teams['team_id'].tolist()

teams.head()

,team_id,team_name
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M


In [14]:
def getTeamID(name):
    return teams[teams['team_name'] == name].values[0][0]

In [15]:
def totalwinsr(team_id,year):
    return len(rsc[(rsc['wteam']==team_id) & (rsc['season']<year)])

In [16]:
def teamname(team_id):
    return teams[teams['team_id'] == team_id].values[0][1]

In [17]:
def totalwinst(team_id,year):
    return len(tourc[(tourc['wteam']==team_id) & (tourc['season']<year)])

In [18]:
def getseed(team_id,year):
    txt=seed[(seed['season']==year)&(seed['team']==team_id)]
    if(len(txt.index)!=0):
        txt=txt.values[0][1]
        match=re.search(r'\d+',txt)
        return int(match.group())
    else: return 25

In [19]:
def totaltrophies(team_id):
    return champlist.count(team_id)

In [20]:
def form(team_id,year):
    last6r=rsc[((rsc['wteam']==team_id) | (rsc['lteam']==team_id)) & (rsc['season']<year)].tail(6)
    last6t=tourc[((tourc['wteam']==team_id) | (tourc['lteam']==team_id)) & (tourc['season']<year)].tail(6)
    rec=float(float(len(last6r[last6r['wteam']==team_id])+len(last6t[last6t['wteam']==team_id]))/12)
    return rec

In [21]:
def tourapps(team_id):
    count=0
    for year in range(1985,2014):
        count+=team_id in np.concatenate([np.asarray(tourc[tourc['season']==year]['wteam'].values),np.asarray(tourc[tourc['season']==year]['lteam'].values)])
    return count

In [22]:
def sos(team_id,year):
    dat=rsc[(rsc['season']==year)]
    def owp(tid):
        lopp=list(dat[dat.wteam==tid].lteam.values)
        wopp=list(dat[dat.lteam==tid].wteam.values)
        allopp=lopp+wopp
        wp=[]
        for i in allopp:
            won=len(dat[(dat.wteam==i)&(dat.lteam!=tid)].index)
            lost=len(dat[(dat.lteam==i)&(dat.wteam!=tid)].index)
            wp.append(float(won)/(won+lost)*100)
        if(len(wp)!=0):
            return float(sum(wp))/len(wp)
        else:
            return 0
    op=owp(team_id)
    oppl=list(dat[dat.wteam==team_id].lteam.values)
    oppw=list(dat[dat.lteam==team_id].wteam.values)
    oppall=oppl+oppw
    oop=[]
    for j in oppall:
        oop.append(owp(j))
    if(len(oop)!=0):
        oop=float(sum(oop))/len(oop)
    else:
        oop=0
    SOS=(2*op+oop)/3
    

    #lis=list(np.concatenate([np.asarray(dat['wteam']==team_id),np.asarray(dat['lteam']==team_id)]))
    #flist=list(set(teamlist).intersection(lis))
    #OOR=[]
    #oppr=[]
    #for tid in flist:
        #OR=[]
        #newlist=[x for x in flist if x!=tid]
        #for i in newlist:
           # won=len(dat[(dat.wteam==i)&(dat.lteam!=tid)].index)
           # lost=len(dat[(dat.lteam==i)&(dat.wteam!=tid)].index)
           # if won+lost!=0:
             #   OR.append(round(float(won/(won+lost)),2))
            #else:
           #     continue
        #if tid==team_id:
        #    oppr.append(sum(OR)/float(len(OR)))
        #else:
           # OOR.append(sum(OR)/float(len(OR)))
    #OOR=sum(OOR)/float(len(OOR))
    #SOS=(2*oppr[0]+OOR)/3
    return SOS
    

In [23]:
def seasonstat(team_id,year):
    games_won=rsc[(rsc['season']==year)&(rsc['wteam']==team_id)]
    games_lost=rsc[(rsc['season']==year)&(rsc['lteam']==team_id)]
    totalpoints = games_won['wscore'].sum()+games_lost['lscore'].sum()
    points_conceded=games_won['lscore'].sum()+games_lost['wscore'].sum()
    wins=len(games_won.index)
    seed=getseed(team_id,year)#seed in the tournament
    whr=totalwinsr(team_id,year)#total regular season wins in history
    wht=totalwinst(team_id,year)#total tournament wins in history
    rec=round(form(team_id,year),2)#overall performance in last 6 regular and tournament games
    trophies=totaltrophies(team_id)#total tournaments won by the team
    numgames=len(games_won.index)+len(games_lost.index)
    games_dwon=rsd[(rsd['season']==year)&(rsd['wteam']==team_id)]
    games_dlost=rsd[(rsd['season']==year)&(rsd['lteam']==team_id)]
    totaldgames=len(games_dwon.index)+len(games_dlost.index)
    count=0
    for year in range(1985,2014):
        count+=team_id in np.concatenate([np.asarray(tourc[tourc['season']==year]['wteam'].values),np.asarray(tourc[tourc['season']==year]['lteam'].values)])
    app=count
    strength_of_schedule=round(sos(team_id,year),2)
    if(totaldgames==0):
        fg=0
        to=0
        fg3=0
        rb=0
        stl=0
        assist=0
        tp=0
        pc=0
    else:
        fg=round(float((games_dwon['wfgm'].sum()+games_dlost['lfgm'].sum()))/totaldgames,2)#fieldgoals
        to=round(float((games_dwon['wto'].sum()+games_dlost['lto'].sum()))/totaldgames,2)#turnovers
        fg3=round(float((games_dwon['wfgm3'].sum()+games_dlost['lfgm3'].sum()))/totaldgames,2)#fieldgoals3pointers
        rb=round(float((games_dwon['wor'].sum()+games_dlost['lor'].sum()))/totaldgames,2)#rebounds
        stl=round(float((games_dwon['wstl'].sum()+games_dlost['lstl'].sum()))/totaldgames,2)#steals
        assist=round(float((games_dwon['wast'].sum()+games_dlost['last'].sum()))/totaldgames,2)#assists
        tp=round(float(totalpoints)/totaldgames,2)#totalpointsscored
        pc=round(float(points_conceded)/totaldgames,2)#totalpointsconceded
        #allaverage
    return [wins,seed,whr,wht,rec,trophies,app,fg,to,fg3,rb,stl,assist,tp,pc,strength_of_schedule]
    
                   

In [521]:
#seasonstat(1228,2014)

In [24]:
def diffteams(id_1, id_2, year):
    team_1 = seasonstat(id_1, year)
    team_2 = seasonstat(id_2, year)
    diff = [a - b for a, b in zip(team_1, team_2)]
    return diff

In [ ]:
def h2h(team1,team2):
    pass

In [25]:
def predict(team_1, team_2, home):
    diff = [a - b for a, b in zip(team_1, team_2)]
    diff.append(home)
    return model.predict([diff])

In [30]:
sample_sub = pd.read_csv('/home/nishant/Downloads/Data/sample_submission_2015.csv')
sample_sub.head()

,id,pred
0,2015_1107_1112,0.5
1,2015_1107_1116,0.5
2,2015_1107_1124,0.5
3,2015_1107_1125,0.5
4,2015_1107_1129,0.5


In [50]:
def finalprediction():
    results = [[0,0]]*len(sample_sub.index)
    for index, row in sample_sub.iterrows():
        match = row['id']
        year = match[0:4]
        team1 = match[5:9]
        team2= match[10:14]
        team1_feat = seasonstat(int(team1), int(year))
        team2_feat = seasonstat(int(team2), int(year))
        pred = predict(team1_feat, team2_feat, 0)
        results[index][0] = match
        results[index][1] = pred[0]
    results = pd.np.array(results)
    title = [[0,0]]
    title[0][0] = 'id'
    title[0][1] = 'pred'
    with open("/home/nishant/Desktop/predictions_2015.csv", "wb") as f:
        writer = csv.writer(f)
        writer.writerows(title)
        writer.writerows(results)

In [40]:
model=cp.load(open('/home/nishant/Desktop/model.pkl'))
finalprediction()

[[0, 0], [0, 0]]